# PyTorch

## Installation

In [9]:
!pip install Google-Colab-Transfer

In [10]:
!pip install SimpleITK

In [11]:
!pip install torch torchvision

In [12]:
!pip install git+https://github.com/ncullen93/torchsample

  Cloning https://github.com/ncullen93/torchsample to /tmp/pip-req-build-yekd39rv
  Running command git clone -q https://github.com/ncullen93/torchsample /tmp/pip-req-build-yekd39rv
  Created wheel for torchsample: filename=torchsample-0.1.3-cp36-none-any.whl size=43417 sha256=f044deebf059a4706b5bd0c2b84488602b01ffebfce9fcb533fedac2bc213ce1
  Stored in directory: /tmp/pip-ephem-wheel-cache-csqmxcxq/wheels/88/c7/72/14cd9a173eed1e29d0b17d866e7d9ee511d31a834aedd27489
Successfully built torchsample


## Data nomenclature

Patients

In [0]:
def get_patient_indices():
  patient_indices = [1,2,5,6,8,10,14,16,18,19,21,22,23,24,25,26,27,28,29,30]

  return patient_indices

Folders

In [0]:
def get_data_folder():
  data_folder = 'data/'

  return data_folder

def get_image_folder():
  image_folder = get_data_folder() + 'imagesTr/'

  return image_folder

def get_ground_truth_folder():
  ground_truth_folder = get_data_folder() + 'labelsTr/'

  return ground_truth_folder

Files

In [0]:
def get_file_extension():
  file_extension = '.nii.gz'

  return file_extension

def get_image_file_name(patient_no, modality_no=0):
  image_modality_convention = '_{:04.0f}'.format(modality_no)
  image_file_name = get_image_folder() + 'patientID' + str(patient_no) + image_modality_convention + get_file_extension()

  return  image_file_name

def get_ground_truth_file_name(patient_no):
  ground_truth_file_name = get_ground_truth_folder() +'patientID' + str(patient_no) + get_file_extension()

  return  ground_truth_file_name

## Import data

In [16]:
import colab_transfer

colab_transfer.mount_google_drive()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
import colab_transfer

from pathlib import Path

for folder_name in [ get_image_folder(), get_ground_truth_folder() ]:
  Path(folder_name).mkdir(parents=True, exist_ok=True)

for patient_no in get_patient_indices():
  for file_name in [ get_image_file_name(patient_no), get_ground_truth_file_name(patient_no) ]:
    
    colab_transfer.copy_file(file_name, 
              source=colab_transfer.get_path_to_home_of_google_drive(),
              destination=colab_transfer.get_path_to_home_of_local_machine())

Copying /content/drive/My Drive/data/imagesTr/patientID1_0000.nii.gz to /content/data/imagesTr/patientID1_0000.nii.gz
Copying /content/drive/My Drive/data/labelsTr/patientID1.nii.gz to /content/data/labelsTr/patientID1.nii.gz
Copying /content/drive/My Drive/data/imagesTr/patientID2_0000.nii.gz to /content/data/imagesTr/patientID2_0000.nii.gz
Copying /content/drive/My Drive/data/labelsTr/patientID2.nii.gz to /content/data/labelsTr/patientID2.nii.gz
Copying /content/drive/My Drive/data/imagesTr/patientID5_0000.nii.gz to /content/data/imagesTr/patientID5_0000.nii.gz
Copying /content/drive/My Drive/data/labelsTr/patientID5.nii.gz to /content/data/labelsTr/patientID5.nii.gz
Copying /content/drive/My Drive/data/imagesTr/patientID6_0000.nii.gz to /content/data/imagesTr/patientID6_0000.nii.gz
Copying /content/drive/My Drive/data/labelsTr/patientID6.nii.gz to /content/data/labelsTr/patientID6.nii.gz
Copying /content/drive/My Drive/data/imagesTr/patientID8_0000.nii.gz to /content/data/imagesTr/p

## Tests

In [0]:
root_folder = '/content/'

In [0]:
patient_indices = get_patient_indices()
patient_no = patient_indices[0]

In [21]:
import numpy as np
import SimpleITK as sitk

image_name = root_folder + get_image_file_name(patient_no)

image = sitk.ReadImage(image_name)
print(image.GetSize())

v = sitk.GetArrayViewFromImage(image)

np.unique(v)

(512, 512, 96)


array([-1200, -1199, -1198, ...,  1235,  1251,  1264], dtype=int32)

In [22]:
import numpy as np
import SimpleITK as sitk

image_name = root_folder + get_ground_truth_file_name(patient_no)

image = sitk.ReadImage(image_name)
print(image.GetSize())

v = sitk.GetArrayViewFromImage(image)

np.unique(v)

(512, 512, 96)


array([  0, 255], dtype=uint8)

## Down-sample images

Reference: https://github.com/jonasteuwen/SimpleITK-examples/blob/master/examples/resample_isotropically.py

In [0]:
# https://github.com/SimpleITK/SlicerSimpleFilters/blob/master/SimpleFilters/SimpleFilters.py
_SITK_INTERPOLATOR_DICT = {
    'nearest': sitk.sitkNearestNeighbor,
    'linear': sitk.sitkLinear,
    'gaussian': sitk.sitkGaussian,
    'label_gaussian': sitk.sitkLabelGaussian,
    'bspline': sitk.sitkBSpline,
    'hamming_sinc': sitk.sitkHammingWindowedSinc,
    'cosine_windowed_sinc': sitk.sitkCosineWindowedSinc,
    'welch_windowed_sinc': sitk.sitkWelchWindowedSinc,
    'lanczos_windowed_sinc': sitk.sitkLanczosWindowedSinc
}

In [0]:
import SimpleITK as sitk
import os
from glob import glob
from tqdm import tqdm
import numpy as np

def resample_sitk_image(sitk_image, spacing=None, interpolator=None,
                        fill_value=0):
    """Resamples an ITK image to a new grid. If no spacing is given,
    the resampling is done isotropically to the smallest value in the current
    spacing. This is usually the in-plane resolution. If not given, the
    interpolation is derived from the input data type. Binary input
    (e.g., masks) are resampled with nearest neighbors, otherwise linear
    interpolation is chosen.
    Parameters
    ----------
    sitk_image : SimpleITK image or str
      Either a SimpleITK image or a path to a SimpleITK readable file.
    spacing : tuple
      Tuple of integers
    interpolator : str
      Either `nearest`, `linear` or None.
    fill_value : int
    Returns
    -------
    SimpleITK image.
    """

    if isinstance(sitk_image, str):
        sitk_image = sitk.ReadImage(sitk_image)
    num_dim = sitk_image.GetDimension()

    if not interpolator:
        interpolator = 'linear'
        pixelid = sitk_image.GetPixelIDValue()

        if pixelid not in [1, 2, 4]:
            raise NotImplementedError(
                'Set `interpolator` manually, '
                'can only infer for 8-bit unsigned or 16, 32-bit signed integers')
        if pixelid == 1: #  8-bit unsigned int
            interpolator = 'nearest'

    orig_pixelid = sitk_image.GetPixelIDValue()
    orig_origin = sitk_image.GetOrigin()
    orig_direction = sitk_image.GetDirection()
    orig_spacing = np.array(sitk_image.GetSpacing())
    orig_size = np.array(sitk_image.GetSize(), dtype=np.int)

    if not spacing:
        min_spacing = orig_spacing.min()
        new_spacing = [min_spacing]*num_dim
    else:
        new_spacing = [float(s) for s in spacing]

    assert interpolator in _SITK_INTERPOLATOR_DICT.keys(),\
        '`interpolator` should be one of {}'.format(_SITK_INTERPOLATOR_DICT.keys())

    sitk_interpolator = _SITK_INTERPOLATOR_DICT[interpolator]

    new_size = orig_size*(orig_spacing/new_spacing)
    new_size = np.ceil(new_size).astype(np.int) #  Image dimensions are in integers
    new_size = [int(s) for s in new_size] #  SimpleITK expects lists, not ndarrays

    resample_filter = sitk.ResampleImageFilter()

    resampled_sitk_image = resample_filter.Execute(sitk_image,
                                                   new_size,
                                                   sitk.Transform(),
                                                   sitk_interpolator,
                                                   orig_origin,
                                                   new_spacing,
                                                   orig_direction,
                                                   fill_value,
                                                   orig_pixelid)

    return resampled_sitk_image


Down-sample for faster checks.

In [0]:
num_dim = 3
new_spacing = [4]*num_dim

In [26]:
for patient_no in get_patient_indices():
  
  file_name = get_ground_truth_file_name(patient_no)     
  
  print(file_name)

  original_image = sitk.ReadImage(file_name)

  # Copy information w.r.t. original spacing
  original_intensity_image = sitk.ReadImage(get_image_file_name(patient_no))
  original_image.CopyInformation(original_intensity_image)

  resampled_image = resample_sitk_image(original_image,
                                        spacing=new_spacing)

  print(original_image.GetSize())
  print(resampled_image.GetSize())

  sitk.WriteImage(resampled_image, file_name) 

data/labelsTr/patientID1.nii.gz
(512, 512, 96)
(89, 89, 36)
data/labelsTr/patientID2.nii.gz
(512, 512, 81)
(100, 100, 41)
data/labelsTr/patientID5.nii.gz
(512, 512, 95)
(97, 97, 38)
data/labelsTr/patientID6.nii.gz
(512, 512, 88)
(84, 84, 33)
data/labelsTr/patientID8.nii.gz
(512, 512, 123)
(94, 94, 50)
data/labelsTr/patientID10.nii.gz
(512, 512, 110)
(89, 89, 44)
data/labelsTr/patientID14.nii.gz
(512, 512, 95)
(73, 73, 38)
data/labelsTr/patientID16.nii.gz
(512, 512, 91)
(102, 102, 46)
data/labelsTr/patientID18.nii.gz
(512, 512, 111)
(88, 88, 45)
data/labelsTr/patientID19.nii.gz
(512, 512, 90)
(86, 86, 36)
data/labelsTr/patientID21.nii.gz
(512, 512, 257)
(100, 100, 65)
data/labelsTr/patientID22.nii.gz
(512, 512, 101)
(93, 93, 41)
data/labelsTr/patientID23.nii.gz
(512, 512, 240)
(85, 85, 60)
data/labelsTr/patientID24.nii.gz
(512, 512, 123)
(82, 82, 62)
data/labelsTr/patientID25.nii.gz
(512, 512, 235)
(77, 77, 59)
data/labelsTr/patientID26.nii.gz
(512, 512, 114)
(98, 98, 46)
data/labelsTr/

In [27]:
for patient_no in get_patient_indices():
  
  file_name = get_image_file_name(patient_no)

  print(file_name)

  original_image = sitk.ReadImage(file_name)
  resampled_image = resample_sitk_image(original_image,
                                        spacing=new_spacing)

  print(original_image.GetSize())
  print(resampled_image.GetSize())

  sitk.WriteImage(resampled_image, file_name)

data/imagesTr/patientID1_0000.nii.gz
(512, 512, 96)
(89, 89, 36)
data/imagesTr/patientID2_0000.nii.gz
(512, 512, 81)
(100, 100, 41)
data/imagesTr/patientID5_0000.nii.gz
(512, 512, 95)
(97, 97, 38)
data/imagesTr/patientID6_0000.nii.gz
(512, 512, 88)
(84, 84, 33)
data/imagesTr/patientID8_0000.nii.gz
(512, 512, 123)
(94, 94, 50)
data/imagesTr/patientID10_0000.nii.gz
(512, 512, 110)
(89, 89, 44)
data/imagesTr/patientID14_0000.nii.gz
(512, 512, 95)
(73, 73, 38)
data/imagesTr/patientID16_0000.nii.gz
(512, 512, 91)
(102, 102, 46)
data/imagesTr/patientID18_0000.nii.gz
(512, 512, 111)
(88, 88, 45)
data/imagesTr/patientID19_0000.nii.gz
(512, 512, 90)
(86, 86, 36)
data/imagesTr/patientID21_0000.nii.gz
(512, 512, 257)
(100, 100, 65)
data/imagesTr/patientID22_0000.nii.gz
(512, 512, 101)
(93, 93, 41)
data/imagesTr/patientID23_0000.nii.gz
(512, 512, 240)
(85, 85, 60)
data/imagesTr/patientID24_0000.nii.gz
(512, 512, 123)
(82, 82, 62)
data/imagesTr/patientID25_0000.nii.gz
(512, 512, 235)
(77, 77, 59)
d

[DONE] check label images after information was copied, to ensure orientation is correctly interpreted by ITK-SNAP.

## Binarize label maps

nnUNet wants consecutive labels: 0, 1, etc.

Originally, the label map in the CHAOS challenge contains only two labels:
- 0 (background),
- 255 (region of interest).

We can simply binarize the label map.


In [28]:
import SimpleITK as sitk
import numpy as np

for patient_no in get_patient_indices():
  print('Patient n°{}'.format(patient_no))

  input_image_name = get_ground_truth_file_name(patient_no)
  input_image = sitk.ReadImage(input_image_name)
  print('Image size: {}'.format(input_image.GetSize()))

  v = sitk.GetArrayFromImage(input_image)
  
  labels = np.unique(v)
  print('Labels: {}'.format(labels))

  max_val = max(labels)
  median_val = max_val/2

  print(median_val)

  # Binarize

  binarized_v = np.zeros(v.shape, v.dtype)
  binarized_v[v>median_val] = 1

  labels = np.unique(binarized_v)
  print('Labels: {}'.format(labels))  

  output_image = sitk.GetImageFromArray(binarized_v)
  
  # Copy meta-data
  output_image.CopyInformation(input_image)

  output_image_name = input_image_name
  sitk.WriteImage(output_image, output_image_name)
  print('Image size: {}'.format(output_image.GetSize()))
  

Patient n°1
Image size: (89, 89, 36)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: (89, 89, 36)
Patient n°2
Image size: (100, 100, 41)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: (100, 100, 41)
Patient n°5
Image size: (97, 97, 38)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: (97, 97, 38)
Patient n°6
Image size: (84, 84, 33)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: (84, 84, 33)
Patient n°8
Image size: (94, 94, 50)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: (94, 94, 50)
Patient n°10
Image size: (89, 89, 44)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: (89, 89, 44)
Patient n°14
Image size: (73, 73, 38)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: (73, 73, 38)
Patient n°16
Image size: (102, 102, 46)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: (102, 102, 46)
Patient n°18
Image size: (88, 88, 45)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: (88, 88, 45)
Patient n°19
Image size: (86, 86, 36)
Labels: [  0 255]
127.5
Labels: [0 1]
Image size: 

## nnUNet

### Installation

In [29]:
#!git clone https://github.com/MIC-DKFZ/nnUNet.git
!git clone https://github.com/woctezuma/nnUNet.git
%cd nnUNet/
!git checkout google-colab
!pip install -r requirements.txt
!pip install -e .
%cd nnunet/

Cloning into 'nnUNet'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 785 (delta 20), reused 28 (delta 12), pack-reused 744
Receiving objects: 100% (785/785), 232.37 KiB | 94.00 KiB/s, done.
Resolving deltas: 100% (548/548), done.
/content/nnUNet
Branch 'google-colab' set up to track remote branch 'google-colab' from 'origin'.
Switched to a new branch 'google-colab'
     |████████████████████████████████| 153kB 53.3MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 7.1MB 22.9MB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
  Created wheel for dicom2nifti: filename=dicom2nifti-2.1.9-cp36-none-any.whl size=41384 sha256=ef49269161fd143e8ea9dd5e094ae5641a7e60b25ee0ab8f9b3b9291ad11093f
  Stored in directory: /root/.cache/pip/wheels/6d/02/2f/e94223dcbdb566149e738087e869d9779fd3ce91ac61aa3baa
  Created wheel for medpy: filename=Me

Obtaining file:///content/nnUNet
  Running setup.py develop for nnunet
/content/nnUNet/nnunet


### Edit paths.py

In [0]:
## Change lines with os.environ() in paths.py
base = "/content/nnUNet_base/"
preprocessing_output_dir = "/content/nnUNet_preprocessed/"
network_training_output_dir = "/content/RESULTS_FOLDER/"

# NB : if you use the google-colab branch in the forked repository, then you do
#      not have to edit paths.py by hand.

In [0]:
from pathlib import Path

Path(preprocessing_output_dir).mkdir(exist_ok=True)

# NB : if you use the google-colab branch in the forked repository, then you do
#      not have to create this directory by hand.

In [32]:
from batchgenerators.utilities.file_and_folder_operations import join

raw_dataset_dir = join(base, "nnUNet_raw")
splitted_4d_output_dir = join(base, "nnUNet_raw_splitted")

print(raw_dataset_dir)
print(splitted_4d_output_dir)

/content/nnUNet_base/nnUNet_raw
/content/nnUNet_base/nnUNet_raw_splitted


### Create repositories

In [0]:
!python paths.py

### Copy data to the right folder

In [0]:
!cp -r /content/data /content/nnUNet_base/nnUNet_raw_splitted/Task00_MY_DATASET

## Edit dataset.json at the root of Task00_MY_DATASET/

Reference: https://github.com/MIC-DKFZ/nnUNet/blob/master/nnunet/dataset_conversion/LiverTumorSegmentationChallenge.py

In [35]:
output_folder = "/content/nnUNet_base/nnUNet_raw_splitted/Task00_MY_DATASET/"

train_ids = [
             'patientID{}'.format(patient_no) 
             for patient_no in get_patient_indices()
]

print(train_ids)

test_ids = [
            
]


['patientID1', 'patientID2', 'patientID5', 'patientID6', 'patientID8', 'patientID10', 'patientID14', 'patientID16', 'patientID18', 'patientID19', 'patientID21', 'patientID22', 'patientID23', 'patientID24', 'patientID25', 'patientID26', 'patientID27', 'patientID28', 'patientID29', 'patientID30']


Decrease the number of patients for faster check of the program.

In [36]:
faster_check = True
num_patients = 5

if faster_check:
  train_ids = train_ids[:5]

  print(train_ids)

['patientID1', 'patientID2', 'patientID5', 'patientID6', 'patientID8']


In [0]:
import os
import json
from collections import OrderedDict

json_dict = OrderedDict()
json_dict['name'] = "CHAOS"
json_dict['description'] = "Combined (CT-MR) Healthy Abdominal Organ Segmentation"
json_dict['tensorImageSize'] = "3D"
json_dict['reference'] = "see challenge website"
json_dict['licence'] = "see challenge website"
json_dict['release'] = "0.0"
json_dict['modality'] = {
    "0": "CT"
}

json_dict['labels'] = {
    "0": "background",
    "1": "liver",
}

json_dict['numTraining'] = len(train_ids)
json_dict['numTest'] = len(test_ids)
json_dict['training'] = [{'image': "./imagesTr/%s.nii.gz" % i, "label": "./labelsTr/%s.nii.gz" % i} for i in train_ids]
json_dict['test'] = ["./imagesTs/%s.nii.gz" % i for i in test_ids]

with open(os.path.join(output_folder, "dataset.json"), 'w') as f:
    json.dump(json_dict, f, indent=4, sort_keys=True)

## How many processes (pl, pf) on Google Colab?

Apparently, 2.


In [38]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits 

## Run pre-processing

In [0]:
!python experiment_planning/plan_and_preprocess_task.py -h

Please cite the following paper when using nnUNet:

Isensee, Fabian, et al. "nnU-Net: Breaking the Spell on Successful Medical Image Segmentation." arXiv preprint arXiv:1904.08128 (2019).

If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet
usage: plan_and_preprocess_task.py [-h] -t TASK [-pl PROCESSES_LOWRES]
                                   [-pf PROCESSES_FULLRES] [-o OVERRIDE]
                                   [-s USE_SPLITTED]
                                   [-no_preprocessing NO_PREPROCESSING]

optional arguments:
  -h, --help            show this help message and exit
  -t TASK, --task TASK  task name. There must be a matching folder in
                        raw_dataset_dir
  -pl PROCESSES_LOWRES, --processes_lowres PROCESSES_LOWRES
                        number of processes used for preprocessing 3d_lowres
                        data, image splitting and image cropping Default: 8.
                        The distinctio

In [39]:
run_pre_processing = False

if faster_check:
  # Force running the pre-processing
  run_pre_processing = True

if run_pre_processing:
  ## Default: -pl 8 -pf 3
  !OMP_NUM_THREADS=1 python experiment_planning/plan_and_preprocess_task.py -t Task00_MY_DATASET -pl 2 -pf 2

Please cite the following paper when using nnUNet:

Isensee, Fabian, et al. "nnU-Net: Breaking the Spell on Successful Medical Image Segmentation." arXiv preprint arXiv:1904.08128 (2019).

If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet
patientID1
patientID2
before crop: (1, 36, 89, 89) after crop: (1, 36, 88, 88) spacing: [4. 4. 4.] 

before crop: (1, 41, 100, 100) after crop: (1, 41, 100, 100) spacing: [4. 4. 4.] 

patientID5
patientID6
before crop: (1, 38, 97, 97) after crop: (1, 38, 97, 97) spacing: [4. 4. 4.] 

before crop: (1, 33, 84, 84) after crop: (1, 33, 84, 84) spacing: [4. 4. 4.] 

patientID8
before crop: (1, 50, 94, 94) after crop: (1, 49, 94, 94) spacing: [4. 4. 4.] 

Are we using the nonzero maks for normalizaion? OrderedDict([(0, False)])
the median shape of the dataset is  [38. 94. 94.]
the max shape in the dataset is  [ 49. 100. 100.]
the min shape in the dataset is  [33. 84. 84.]
we don't want feature maps smalle

Save to Google Drive

In [0]:
local_data_path = '/content/nnUNet_preprocessed/Task00_MY_DATASET/'
gdrive_data_path = '/content/drive/My Drive/nnUNet_preprocessed/Task00_MY_DATASET/'

In [0]:
import colab_transfer

if not faster_check:
  colab_transfer.copy_folder_structure(source=local_data_path,
                                       destination=gdrive_data_path)

Load from Google Drive

In [0]:
import colab_transfer

if not faster_check:
  colab_transfer.copy_folder_structure(source=gdrive_data_path,
                                       destination=local_data_path)

In [42]:
!ls /content/nnUNet_preprocessed/Task00_MY_DATASET/nnUNet_2D_stage0/

patientID1.npz	patientID2.npz	patientID5.npz	patientID6.npz	patientID8.npz
patientID1.pkl	patientID2.pkl	patientID5.pkl	patientID6.pkl	patientID8.pkl


## Training

In [0]:
!python run/run_training.py -h

Please cite the following paper when using nnUNet:

Isensee, Fabian, et al. "nnU-Net: Breaking the Spell on Successful Medical Image Segmentation." arXiv preprint arXiv:1904.08128 (2019).

If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet
usage: run_training.py [-h] [-val] [-c] [-p P] [-u UNPACK_DATA] [--ndet]
                       [--npz] [--find_lr] [--valbest] [--fp16]
                       network network_trainer task fold

positional arguments:
  network
  network_trainer
  task
  fold                  0, 1, ..., 5 or 'all'

optional arguments:
  -h, --help            show this help message and exit
  -val, --validation_only
                        use this if you want to only run the validation
  -c, --continue_training
                        use this if you want to continue a training
  -p P                  plans identifier
  -u UNPACK_DATA, --unpack_data UNPACK_DATA
                        Leave it as 1, development only


In [0]:
!OMP_NUM_THREADS=1 python run/run_training.py 3d_fullres nnUNetTrainer Task00_MY_DATASET all --ndet

Please cite the following paper when using nnUNet:

Isensee, Fabian, et al. "nnU-Net: Breaking the Spell on Successful Medical Image Segmentation." arXiv preprint arXiv:1904.08128 (2019).

If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet
###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainer.nnUNetTrainer'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'CT'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region OrderedDict([((1,), True)])
min_region_size_per_class OrderedDict([(1, 699477.76)])
min_size_per_class OrderedDict([(1, 699477.76)])
normalization_schemes OrderedDict([(0, 'CT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [3, 4, 4], 'patch_size': array([40, 96, 96]), 'median_patient_size_in_voxels': array([38, 94, 94]), 'current_spacing': 

## Save the checkpoints

In [0]:
import colab_transfer

colab_transfer.copy_folder_structure(
    '/content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/',
    '/content/drive/My Drive/nnUNet_results/'
)

['/content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/all', '/content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/plans.pkl']
['/content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/plans.pkl']
/content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/all
all/
debug.json
Copying debug.json
all/
training_log_2019_12_17_12_00_45.txt
Copying training_log_2019_12_17_12_00_45.txt
all/
model_latest.model
Copying model_latest.model
all/
training_log_2019_12_22_17_38_11.txt
Copying training_log_2019_12_22_17_38_11.txt
all/
model_latest.model.pkl
Copying model_latest.model.pkl
all/
model_best.model.pkl
Copying model_best.model.pkl
all/
progress.png
Copying progress.png
all/
model_best.model
Copying model_best.model
/content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/plans.pkl
plans.pkl
Copying plans.pkl


## Load the checkpoints

In [43]:
import colab_transfer

colab_transfer.copy_folder_structure(
    '/content/drive/My Drive/nnUNet_results/',
    '/content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/',
)

Files: ['/content/drive/My Drive/nnUNet_results/plans.pkl']
Folders: {'/content/drive/My Drive/nnUNet_results/all'}
Copying /content/drive/My Drive/nnUNet_results/plans.pkl to /content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/plans.pkl
Copying /content/drive/My Drive/nnUNet_results/all/debug.json to /content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/all/debug.json
Copying /content/drive/My Drive/nnUNet_results/all/training_log_2019_12_17_12_00_45.txt to /content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/all/training_log_2019_12_17_12_00_45.txt
Copying /content/drive/My Drive/nnUNet_results/all/model_latest.model to /content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/all/model_latest.model
Copying /content/drive/My Drive/nnUNet_results/all/training_log_2019_12_22_17_38_11.txt to /content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/all/training_lo

## Continue training

In [0]:
!OMP_NUM_THREADS=1 python run/run_training.py 3d_fullres nnUNetTrainer Task00_MY_DATASET all --ndet -c

Please cite the following paper when using nnUNet:

Isensee, Fabian, et al. "nnU-Net: Breaking the Spell on Successful Medical Image Segmentation." arXiv preprint arXiv:1904.08128 (2019).

If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet
###############################################
I am running the following nnUNet: 3d_fullres
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainer.nnUNetTrainer'>
For that I will be using the following configuration:
num_classes:  1
modalities:  {0: 'CT'}
use_mask_for_norm OrderedDict([(0, False)])
keep_only_largest_region OrderedDict([((1,), True)])
min_region_size_per_class OrderedDict([(1, 699477.76)])
min_size_per_class OrderedDict([(1, 699477.76)])
normalization_schemes OrderedDict([(0, 'CT')])
stages...

stage:  0
{'batch_size': 2, 'num_pool_per_axis': [3, 4, 4], 'patch_size': array([40, 96, 96]), 'median_patient_size_in_voxels': array([38, 94, 94]), 'current_spacing': 

## Inference

In [0]:
!OMP_NUM_THREADS=1 python inference/predict_simple.py -h

Please cite the following paper when using nnUNet:

Isensee, Fabian, et al. "nnU-Net: Breaking the Spell on Successful Medical Image Segmentation." arXiv preprint arXiv:1904.08128 (2019).

If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet
usage: predict_simple.py [-h] -i INPUT_FOLDER -o OUTPUT_FOLDER -t TASK_NAME
                         [-tr NNUNET_TRAINER] [-m MODEL] [-p PLANS_IDENTIFIER]
                         [-f FOLDS [FOLDS ...]] [-z] [-l LOWRES_SEGMENTATIONS]
                         [--part_id PART_ID] [--num_parts NUM_PARTS]
                         [--num_threads_preprocessing NUM_THREADS_PREPROCESSING]
                         [--num_threads_nifti_save NUM_THREADS_NIFTI_SAVE]
                         [--tta TTA] [--overwrite_existing OVERWRITE_EXISTING]

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT_FOLDER, --input_folder INPUT_FOLDER
                        Must contain all modal

In [0]:
INPUT_FOLDER='/content/nnUNet_base/nnUNet_raw_splitted/Task00_MY_DATASET/imagesTr/'

In [0]:
!ls $INPUT_FOLDER

patientID10_0000.nii.gz  patientID21_0000.nii.gz  patientID28_0000.nii.gz
patientID1_0000.nii.gz	 patientID22_0000.nii.gz  patientID29_0000.nii.gz
patientID14_0000.nii.gz  patientID23_0000.nii.gz  patientID30_0000.nii.gz
patientID16_0000.nii.gz  patientID24_0000.nii.gz  patientID5_0000.nii.gz
patientID18_0000.nii.gz  patientID25_0000.nii.gz  patientID6_0000.nii.gz
patientID19_0000.nii.gz  patientID26_0000.nii.gz  patientID8_0000.nii.gz
patientID2_0000.nii.gz	 patientID27_0000.nii.gz


In [0]:
OUTPUT_FOLDER='/content/data/output/'
!mkdir $OUTPUT_FOLDER

In [0]:
!ls $OUTPUT_FOLDER

In [0]:
!OMP_NUM_THREADS=1 python inference/predict_simple.py -i $INPUT_FOLDER -o $OUTPUT_FOLDER -t Task00_MY_DATASET -f all -tr nnUNetTrainer -m 3d_fullres

Please cite the following paper when using nnUNet:

Isensee, Fabian, et al. "nnU-Net: Breaking the Spell on Successful Medical Image Segmentation." arXiv preprint arXiv:1904.08128 (2019).

If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet
using model stored in  /content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans
emptying cuda cache
loading parameters for folds, ['all']
using the following model files:  ['/content/RESULTS_FOLDER/3d_fullres/Task00_MY_DATASET/nnUNetTrainer__nnUNetPlans/all/model_best.model']
starting preprocessing generator
starting prediction...
preprocessing /content/data/output/patientID1.nii.gz
preprocessing /content/data/output/patientID10.nii.gz
before crop: (1, 36, 89, 89) after crop: (1, 36, 88, 88) spacing: [4. 4. 4.] 

no resampling necessary
no resampling necessary
preprocessing /content/data/output/patientID14.nii.gz
before: {'spacing': array([4., 4., 4.]), 'spacing_transposed': a

In [0]:
import colab_transfer

colab_transfer.copy_folder_structure(
    '/content/data/',
    '/content/drive/My Drive/downsampled_data/',    
)

['/content/data/imagesTr', '/content/data/labelsTr', '/content/data/output']
[]
/content/data/imagesTr
imagesTr/
patientID2_0000.nii.gz
Copying patientID2_0000.nii.gz
imagesTr/
patientID24_0000.nii.gz
Copying patientID24_0000.nii.gz
imagesTr/
patientID5_0000.nii.gz
Copying patientID5_0000.nii.gz
imagesTr/
patientID23_0000.nii.gz
Copying patientID23_0000.nii.gz
imagesTr/
patientID6_0000.nii.gz
Copying patientID6_0000.nii.gz
imagesTr/
patientID18_0000.nii.gz
Copying patientID18_0000.nii.gz
imagesTr/
patientID8_0000.nii.gz
Copying patientID8_0000.nii.gz
imagesTr/
patientID27_0000.nii.gz
Copying patientID27_0000.nii.gz
imagesTr/
patientID10_0000.nii.gz
Copying patientID10_0000.nii.gz
imagesTr/
patientID25_0000.nii.gz
Copying patientID25_0000.nii.gz
imagesTr/
patientID1_0000.nii.gz
Copying patientID1_0000.nii.gz
imagesTr/
patientID22_0000.nii.gz
Copying patientID22_0000.nii.gz
imagesTr/
patientID26_0000.nii.gz
Copying patientID26_0000.nii.gz
imagesTr/
patientID16_0000.nii.gz
Copying patient

## Visualize segmentation results

[DONE] Download data and display segmentation overlayed on CT images with ITK-SNAP.